In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3
%cd ../

In [ ]:
import train
from lib import utils
from lib import evaluation
from lib.dataset import EPICDiff
import matplotlib.pyplot as plt
from train import get_latest_ckpt

In [ ]:
vid = 'P01_104'
# vid = 'P01_09'
# vid = 'P01_01'
hparams = train.get_opts(vid=vid)

In [ ]:
hparams.root = 'data/'

In [ ]:
hparams = train.get_opts(vid=vid)

In [ ]:
hparams.num_workers = 8
hparams.dl_type = 'train'
hparams.num_gpus = 1

In [ ]:
import os

In [ ]:
hparams.model_type = 'neuraldiff'
hparams.ckpt_dst = 'ckpts_bck/dbg'
hparams.logs_dst = 'logs/dbg'
hparams.train.ignore_split = True

In [ ]:
hparams.ckpt_path = get_latest_ckpt(f'ckpts_bck/{vid}-neuraldiff/')
# hparams.ckpt_path = get_latest_ckpt(f'ckpts/{vid}-neuraldiff/')

In [ ]:
if hparams.ckpt_path is None:
    model = train.NeuralDiffSystem(hparams)
else:
    dataset = EPICDiff(vid=vid, cfg=hparams, split='val')
    max_frame_id = max(dataset.img_ids)
    model = train.init_model(hparams.ckpt_path, dataset=None, hparams=hparams, max_frame_id=max_frame_id)
#     model = train.init_model(hparams.ckpt_path, dataset=dataset, hparams=hparams)

In [ ]:
model.setup(0, reset_dataset=True)

In [ ]:
model.hparams.exp_name = f'debug/{vid}-{model.hparams.model_type}'
hparams.exp_name = model.hparams.exp_name

In [ ]:
model.train()
hparams.train_ratio = 200
hparams.num_epochs = 11
trainer = train.init_trainer(hparams, system=model)

In [ ]:
model.val_dataset.split = 'val'
trainer.fit(model)

In [ ]:
import time
t1 = time.time()
model.val_dataset.split = 'all'
model.eval()
model.cuda()
sample_id = model.val_dataset.img_ids[70]
x = evaluation.evaluate_sample(model.val_dataset, model=model, sample_id=sample_id)
t2 = time.time()

In [ ]:
plt.imshow(x['grid'])
plt.axis('off')
plt.show()